# Extremes - Demo

In [11]:
import autoroot
import xarray as xr

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
data_dir = '/pool/datos/reanalisis/era5/anomaly/spain/025/t2max/*.nc'

In [13]:
ds = xr.open_mfdataset(data_dir, engine="netcdf4")

In [14]:
ds

<xarray.Dataset>
Dimensions:    (time: 23011, latitude: 41, longitude: 61)
Coordinates:
  * time       (time) datetime64[ns] 1959-01-01 1959-01-02 ... 2021-12-31
    dayofyear  (time) int32 dask.array<chunksize=(365,), meta=np.ndarray>
  * longitude  (longitude) float32 -10.0 -9.75 -9.5 -9.25 ... 4.25 4.5 4.75 5.0
  * latitude   (latitude) float32 45.0 44.75 44.5 44.25 ... 35.5 35.25 35.0
Data variables:
    t2m        (time, latitude, longitude) float64 dask.array<chunksize=(365, 41, 61), meta=np.ndarray>